In [348]:
import pandas as pd

In [349]:
matches_df = pd.read_csv('matches.csv', index_col=0)

In [350]:
matches_df['opponent'] = matches_df['opponent'].str.split().str[1:].str.join(' ')

In [351]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959 entries, 0 to 28
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1959 non-null   object 
 1   time          1959 non-null   object 
 2   comp          1959 non-null   object 
 3   round         1959 non-null   object 
 4   day           1959 non-null   object 
 5   venue         1959 non-null   object 
 6   result        1958 non-null   object 
 7   gf            1958 non-null   object 
 8   ga            1958 non-null   object 
 9   opponent      1959 non-null   object 
 10  poss          1049 non-null   float64
 11  attendance    939 non-null    float64
 12  captain       1852 non-null   object 
 13  formation     1843 non-null   object 
 14  referee       1955 non-null   object 
 15  match report  1959 non-null   object 
 16  notes         64 non-null     object 
 17  sh            1050 non-null   float64
 18  sot           1050 non-null   

In [352]:
matches_df['date'] = pd.to_datetime(matches_df['date'])

In [353]:
#clening the gf and ga column and making it an int
matches_df['gf'] = matches_df['gf'].str.split().str[0]
matches_df['gf'] = matches_df['gf'].str.split('.').str[0]

matches_df['ga'] = matches_df['ga'].str.split().str[0]
matches_df['ga'] = matches_df['ga'].str.split('.').str[0]

matches_df.dropna(subset=['gf'], inplace=True)
matches_df['gf'] = matches_df['gf'].astype(int)
matches_df['ga'] = matches_df['ga'].astype(int)

In [354]:
matches_df['opp_code'] = matches_df.opponent.astype('category').cat.codes 

In [355]:
matches_df['hour'] = matches_df['time'].str.replace(':.+', '', regex=True).astype(int)

In [356]:
matches_df['round_code'] = matches_df['round'].astype('category').cat.codes 

In [357]:
matches_df['day_code'] = matches_df['date'].dt.dayofweek

In [358]:
#matches_df['target'] = matches_df['result'].astype('category').cat.codes
matches_df['target'] = (matches_df['result'] == 'W').astype(int)

In [359]:
matches_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pkatt,season,team,xg,xga,opp_code,hour,round_code,day_code,target
0,2020-10-08,21:30,WCQ,WCQ — CONMEBOL (M),Thu,Away,L,0,1,Argentina,...,0.0,2022,Ecuador,NaN,NaN,5,21,14,3,0
1,2020-10-13,16:00,WCQ,WCQ — CONMEBOL (M),Tue,Home,W,4,2,Uruguay,...,0.0,2022,Ecuador,NaN,NaN,163,16,14,1,1
2,2020-11-12,16:00,WCQ,WCQ — CONMEBOL (M),Thu,Away,W,3,2,Bolivia,...,1.0,2022,Ecuador,NaN,NaN,18,16,14,3,1
3,2020-11-17,16:00,WCQ,WCQ — CONMEBOL (M),Tue,Home,W,6,1,Colombia,...,0.0,2022,Ecuador,NaN,NaN,34,16,14,1,1
4,2021-06-04,21:30,WCQ,WCQ — CONMEBOL (M),Fri,Away,L,0,2,Brazil,...,0.0,2022,Ecuador,NaN,NaN,21,21,14,4,0


In [360]:
matches_df.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                       int64
ga                       int64
opponent                object
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                   object
sh                     float64
sot                    float64
pk                     float64
pkatt                  float64
season                  object
team                    object
xg                     float64
xga                    float64
opp_code                 int16
hour                     int64
round_code                int8
day_code                 int64
target                   int64
dtype: object

In [361]:
matches_no_na = matches_df.dropna(subset=['poss','sh', 'sot'])
print('len: ', len(matches_no_na['team'].value_counts()))
matches_no_na['team'].value_counts()


len:  32


Spain             59
England           58
Belgium           58
France            57
Denmark           57
Poland            56
Croatia           55
Netherlands       55
Germany           55
Portugal          53
Wales             51
Serbia            49
Switzerland       49
Brazil            44
Argentina         43
Uruguay           42
Ecuador           36
Senegal           21
Cameroon          18
Tunisia           17
Qatar             15
Morocco           14
Mexico            12
United-States     12
Australia         11
Ghana             10
Japan             10
Canada             9
Costa-Rica         8
IR-Iran            6
Korea-Republic     5
Saudi-Arabia       4
Name: team, dtype: int64

In [362]:
from sklearn.ensemble import RandomForestClassifier

In [363]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [364]:
train = matches_no_na[matches_no_na['date'] < '2022-01-01']

In [365]:
test = matches_no_na[matches_no_na['date'] > '2022-01-01']

In [366]:
len(train) +len(test)

1049

In [367]:
predictors = ['opp_code', 'hour', 'day_code']

In [368]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [369]:
preds = rf.predict(test[predictors])

In [370]:
from sklearn.metrics import accuracy_score

In [371]:
acc = accuracy_score(test['target'], preds)

In [372]:
acc

0.5317460317460317

In [373]:
combined = pd.DataFrame(dict(actual = test['target'], prediction = preds))

In [374]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0,1
actual,,
0,30,37
1,22,37


In [375]:
from sklearn.metrics import precision_score

In [376]:
precision_score(test['target'], preds)

0.5